In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.ui.proxyBase", "/user/" + os.environ["JUPYTERHUB_USER"] + "/proxy/4041")
conf.set("spark.driver.memory", "16g")

sc = pyspark.SparkContext(conf = conf)
spark = pyspark.SQLContext.getOrCreate(sc)